#### This notebook we will work in DecisionTreeClassifier


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection  import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import GridSearchCV

%matplotlib inline
SEED = 42

In [13]:
# Read processe file
df = pd.read_csv('../data/processed_balanced_transaction.csv')
df.head()

,accountNumber,acqCountry,availableMoney,cardCVV,cardLast4Digits,cardPresent,creditLimit,currentBalance,customerId,enteredCVV,...,transactionDateTime_day,transactionDateTime_hour,transactionDateTime_minute,transactionDateTime_second,currentExpDate_year,currentExpDate_month,currentExpDate_day,dateOfLastAddressChange_year,dateOfLastAddressChange_month,dateOfLastAddressChange_day
0,173544488,3,3060.76,711,460,1,7500.0,4439.24,173544488,711,...,30,22,45,7,2019,9,1,2014,4,12
1,109517864,3,250.00,912,4842,1,250.0,0.00,109517864,912,...,28,2,14,48,2024,3,1,2014,11,7
2,937106980,3,8272.83,312,92,1,15000.0,6727.17,937106980,312,...,9,12,0,17,2022,4,1,2016,9,16
3,942524230,3,5289.40,379,1683,1,7500.0,2210.60,942524230,379,...,8,4,17,52,2028,11,1,2008,11,16
4,899404344,3,3942.77,279,8069,1,5000.0,1057.23,899404344,279,...,9,11,35,49,2019,5,1,2012,3,22


#### Lets separate Level and features, Scaled features


In [14]:
X, y = df.drop(['isFraud'],axis=1), df['isFraud']

#### Split data into train test 
 We are spliting data into train and test with ratio 30%. Means 30% test and 70% train data

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=SEED)

#### Create a DessionTree classifier and fit all features

In [20]:
dt_classifier = DecisionTreeClassifier(max_depth=5, random_state=SEED)
dt_classifier.fit(X_train, y_train)
y_pred = dt_classifier.predict(X_test)


In [21]:
from sklearn import metrics
target_names = ['Not Fraud', 'Fraud']
print(metrics.classification_report(y_test, y_pred, digits=3, target_names=target_names))

              precision    recall  f1-score   support

   Not Fraud      0.667     0.706     0.686    130734
       Fraud      0.688     0.648     0.667    130907

    accuracy                          0.677    261641
   macro avg      0.678     0.677     0.677    261641
weighted avg      0.678     0.677     0.677    261641



#### Converting to standard scale and fit again

In [22]:
standard_scaler = StandardScaler()
X = standard_scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=SEED)

In [23]:
dt_classifier = DecisionTreeClassifier(max_depth=5, random_state=SEED)
dt_classifier.fit(X_train, y_train)
y_pred = dt_classifier.predict(X_test)

In [24]:
print(metrics.classification_report(y_test, y_pred, digits=3, target_names=target_names))

              precision    recall  f1-score   support

   Not Fraud      0.667     0.706     0.686    130734
       Fraud      0.688     0.648     0.667    130907

    accuracy                          0.677    261641
   macro avg      0.678     0.677     0.677    261641
weighted avg      0.678     0.677     0.677    261641



#### Cross validation
we are going to apply k-fold cross-validation.

it will split the original data set into k subsets and use one of the subsets as the testing set and the remaining as the training sets. This process iterated k times until every subset have been used as the testing set. Since 10-fold cross-validation is the most popular one, we are going to use that one.

In [ ]:
dt_classifier = DecisionTreeClassifier(max_depth=5, random_state=SEED)
cv_scores = cross_val_score(dt_classifier, X_train, y_train, cv=10)
print('Average score: {}'.format(np.mean(cv_scores)))

It looks there is no improvement using cross-validation. 

#### Fit model on Best 20 important features

In [ ]:
## TODO

#### Parameter Tuning

In classification technique, there are some parameters that can be tuned to optimize the classification. 
In DecessionTreeClassifier we can tune 

- Decision tree is max depth (the depth of the tree)
- max feature (the feature used to classify)
- criterion
- splitter

Grid Search explores a range of parameters and finds the best combination of parameters. Then repeat the process several times until the best parameters are discovered. 
lets use grid search to get best params


In [105]:
from sklearn.model_selection import StratifiedKFold

dt_classifier = DecisionTreeClassifier()

parameter_grid = {
                  'criterion': ['gini', 'entropy'],
                  'splitter': ['best', 'random'],
                  'max_depth': [3, 4, 5, 6],
                  'max_features': [15, 20, 25, 30]
                 }

cross_validation = StratifiedKFold(n_splits=10)

grid_search = GridSearchCV(dt_classifier, param_grid=parameter_grid, cv=cross_validation)

grid_search.fit(X_train, y_train)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

best_dt_classifier = grid_search.best_estimator_
best_dt_classifier

Best score: 0.982994960365098
Best parameters: {'criterion': 'entropy', 'max_depth': 5, 'max_features': 20, 'splitter': 'best'}


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=5, max_features=20, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

A little bit improvement in best score. lets fit best model


In [115]:
best_dt_classifier.fit(X_train, y_train)
y_pred = best_dt_classifier.predict(X_test)
accuracy = round(accuracy_score(y_test, y_pred),5)
precision = round(precision_score(y_test, y_pred, zero_division=1),5)
recall = round(recall_score(y_test, y_pred),5)
print('acc: {} precision: {} recal: {}'.format(accuracy, precision, recall))


acc: 0.98253 precision: 1.0 recal: 0.0


#### Save best model 

In [118]:
import pickle

lookup ={
    1 : 'Fraud', 0:'Not Fraud'
}

pickle.dump(best_dt_classifier, open('../saved_models/dtc_model.pkl','wb'))
model = pickle.load(open('../saved_models/dtc_model.pkl','rb'))

#test the model 
pred = model.predict([X_test[0]])
lookup[pred[0]]

'Not Fraud'

#### lets write score in a file

In [120]:
import csv

with open('../data/dtc_scrore.csv', 'w', newline='') as csvfile:
    fieldnames = ['accuracy_score', 'precision_score', 'recall_score']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerow({'accuracy_score': accuracy, 'precision_score':  precision, 'recall_score':recall})


#### Code Reference

- https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html
- https://medium.com/@haydar_ai/learning-data-science-day-22-cross-validation-and-parameter-tuning-b14bcbc6b012
-  https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html#sklearn.model_selection.StratifiedKFold
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html